In [1]:
"""
Student: Dinh Khac Tuyen
ID:20214182

"""
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt

In [2]:
# General EncoderDecoder Architecture
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many 
    other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask,
                            tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [3]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

In [4]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [5]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [6]:
class LayerNorm(nn.Module):
    #"Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [7]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

In [8]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

In [9]:
class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [10]:
class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [11]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

In [12]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [13]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

In [14]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [15]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [16]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

In [17]:
def make_model(src_vocab, tgt_vocab, N=6, 
               d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                             c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model

In [18]:
class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, src, trg=None, pad=0):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if trg is not None:
            self.trg = trg[:, :-1]
            self.trg_y = trg[:, 1:]
            self.trg_mask = \
                self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg_y != pad).data.sum()
    
    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & Variable(
            subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        return tgt_mask

In [19]:
def run_epoch(data_iter, model, loss_compute):
    "Standard Training and Logging Function"
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(batch.src, batch.trg, 
                            batch.src_mask, batch.trg_mask)
        loss = loss_compute(out, batch.trg_y, batch.ntokens)
        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 50 == 1:
            elapsed = time.time() - start
#             print("Epoch Step: %d Loss: %f Tokens per Sec: %f" %
#                     (i, loss / batch.ntokens, tokens / elapsed))
            start = time.time()
            tokens = 0
    return total_loss / total_tokens

In [20]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
        
def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

In [21]:
class LabelSmoothing(nn.Module):
    "Implement label smoothing."
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))

In [24]:
"""
Problem 2.1: Copying list of digit
"""

def data_gen(V, batch, nbatches,sq_len,device):
    "Generate random digit list for a src-tgt copy task."
    for i in range(nbatches):
        data = torch.from_numpy(np.random.randint(1, V, size=(batch, sq_len)))
        start_value = torch.zeros((batch,1),dtype=torch.long)
        src = Variable(data, requires_grad=False)
        # add start_value to the first of tgt
        tgt = Variable(torch.cat((start_value,data),1), requires_grad=False)
        yield Batch(src.to(device), tgt.to(device), 0)

In [25]:
class SimpleLossCompute:
    "A simple loss compute and train function."
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm):
        x = self.generator(x)
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)), 
                              y.contiguous().view(-1)) / norm
        loss.backward()
        if self.opt is not None:
            self.opt.step()
            self.opt.optimizer.zero_grad()
        return loss.data.item() * norm

In [26]:
# Train the simple copy task.
V = 10                      # digit from 1-9, and start_value=0
criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
digit_model = make_model(V, V, N=2)
digit_model.to(device)
model_opt = NoamOpt(digit_model.src_embed[0].d_model, 1, 400,
        torch.optim.Adam(digit_model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
sequence_len=32
for epoch in range(50):
    digit_model.train()
    print("Epoch:{}/50".format(epoch))
    run_epoch(data_gen(V, 30, 20,sequence_len,device), digit_model, 
              SimpleLossCompute(digit_model.generator, criterion, model_opt))
    digit_model.eval()
    print(run_epoch(data_gen(V, 30, 5,sequence_len,device), digit_model, 
                    SimpleLossCompute(digit_model.generator, criterion, None)))

/home/ncl/.conda/envs/ktdinh/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch:0/50
tensor(2.1753, device='cuda:0')
Epoch:1/50
tensor(2.1246, device='cuda:0')
Epoch:2/50
tensor(2.0642, device='cuda:0')
Epoch:3/50
tensor(2.1039, device='cuda:0')
Epoch:4/50
tensor(2.1368, device='cuda:0')
Epoch:5/50
tensor(2.0901, device='cuda:0')
Epoch:6/50
tensor(2.0023, device='cuda:0')
Epoch:7/50
tensor(1.5803, device='cuda:0')
Epoch:8/50
tensor(1.5404, device='cuda:0')
Epoch:9/50
tensor(1.5454, device='cuda:0')
Epoch:10/50
tensor(0.8565, device='cuda:0')
Epoch:11/50
tensor(1.2423, device='cuda:0')
Epoch:12/50
tensor(1.0158, device='cuda:0')
Epoch:13/50
tensor(1.0787, device='cuda:0')
Epoch:14/50
tensor(0.6217, device='cuda:0')
Epoch:15/50
tensor(0.3713, device='cuda:0')
Epoch:16/50
tensor(0.1406, device='cuda:0')
Epoch:17/50
tensor(0.1399, device='cuda:0')
Epoch:18/50
tensor(0.1207, device='cuda:0')
Epoch:19/50
tensor(0.0626, device='cuda:0')
Epoch:20/50
tensor(0.0467, device='cuda:0')
Epoch:21/50
tensor(0.0410, device='cuda:0')
Epoch:22/50
tensor(0.0291, device='cuda:0'

In [30]:
def greedy_decode(model, src, src_mask, sequence_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(sequence_len):
        out = model.decode(memory, src_mask, 
                           Variable(ys), 
                           Variable(subsequent_mask(ys.size(1))
                                    .type_as(src.data)))
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.data[0]
        ys = torch.cat([ys, 
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return ys

digit_model.eval()

c=0
for i in range(1000):
    src_data = torch.randint(1,10,(1,sequence_len))
    src = Variable(src_data, requires_grad=False)
    src_mask = Variable(torch.ones(1, 1, sequence_len))
    out=greedy_decode(digit_model, src.to(device), src_mask.to(device), sequence_len=sequence_len, start_symbol=0).cpu()
    # cut the start_value and compare with src
    out=out[:,1:]
    if torch.equal(src_data,out):
        c+=1
print("Copying success rate:{}".format(c/1000))


Copying success rate:0.947


In [31]:
import random
import re
def preprocess(phrase):
    # handle abbreviation
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    # decapitalize
    phrase = phrase.lower()
    # remove unnecessary symbols
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in symbols:
        phrase = phrase.replace(i," ")
    return phrase

def create_vocab(dataset):
    # Create vocab from dataset
    vocab=[]
    word_max_len=0
    for i,each in enumerate(dataset):
        # First, we have to ignore data_ids that share the same document in the dataset. Because data_ids that share same context are contiguous so we just need to check each['context'] with the previous 
        if i!=0:    
            if each['context'] == dataset[i-1]['context']:   # ignore 
                continue
        doc=preprocess(each['context'])
        words=doc.split()
        for word in words:
            # ignore word contain non-letter characters
            if word.isalpha()==False:
                continue
            if word.isascii() and word not in vocab:
                vocab.append(word)
                if len(word)>word_max_len:
                    word_max_len=len(word)
    vocab_len=len(vocab)
    print("Number of word in vocab:{},{}".format(len(vocab),word_max_len))
    return vocab,word_max_len

from datasets import load_dataset
from pprint import pprint

squad_dataset = load_dataset('squad')
# vocab: list contain all words, word_max_len: len of the longest word.
vocab,word_max_len=create_vocab(squad_dataset['train'])
random.shuffle(vocab)
vocab_train,vocab_val=vocab[:4*len(vocab)//5],vocab[4*len(vocab)//5:]

Reusing dataset squad (/home/ncl/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


Number of word in vocab:64950,27


In [32]:
"""
Problem 2.2: Spelling correction word-level
"""
def char_encode(c):
    # encode char a-z to 2->28, the others: 0 for start_value, 1 for padding
    return ord(c)-ord('a')+2
def word_embed(word,word_len):
    # embed a word to vector of (28), return both embed and embed_noise. The noise is create by double a random character in word
    embed=[]
    embed_noise=[]
    idx=random.randint(0,len(word)-1) # random choose position of character to make noise
    for i in range(len(word)):
        if i==idx:  # duplicate the character at position
            embed_noise.append(char_encode(word[i]))
        embed.append(char_encode(word[i]))
        embed_noise.append(char_encode(word[i]))
    for i in range(len(embed),word_len):
            embed.append(1)
    for i in range(len(embed_noise),word_len):
            embed_noise.append(1)
    return torch.LongTensor(embed),torch.LongTensor(embed_noise)
        

In [33]:
# sets that contain list of data, each data [padded_word,padded_word_with_noise]
word_trainset=[]
word_valset=[]
for each in vocab_train:
    word,word_noise=word_embed(each,word_max_len+1)
    word_trainset.append([word,word_noise])
for each in vocab_val:
    word,word_noise=word_embed(each,word_max_len+1)
    word_valset.append([word,word_noise])

In [51]:
def word_batch_gen(vocab_set, batch,word_len,device):
    """
    batch_gen, generate src and tgt batch for training. 
    The goal is to train model to learn translate from src(orig word with noise) to tgt(orig word with start_value at first)
    """
    # shuffle for each batch
    random.shuffle(vocab_set)
    n_batches=len(vocab_set)//batch
    for batch_idx in range(n_batches):
        ws=torch.LongTensor()  # tensor hold tgt
        ws_noise=torch.LongTensor()    # tensor hold src(src have noise)
        for i in range(batch_idx*batch,batch*(batch_idx+1)):
            ws=torch.cat((ws,vocab_set[i][0].unsqueeze(0)),0)
            ws_noise=torch.cat((ws_noise,vocab_set[i][1].unsqueeze(0)),0)   
        src = Variable(ws_noise, requires_grad=False)
        # start_value for each word, here start_value=0
        start_value=torch.zeros((batch,1),dtype=torch.long)
        # concat start_value to tgt
        tgt = Variable(torch.cat((start_value,ws),1), requires_grad=False)
        yield Batch(src.to(device), tgt.to(device), 0)

In [52]:
V=28 # size of vocab=28 (from a->z +2)
criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
word_model = make_model(V, V, N=2)
word_model.to(device)
model_opt = NoamOpt(word_model.src_embed[0].d_model, 1, 400,
        torch.optim.Adam(word_model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
for epoch in range(100):
    word_model.train()
    print("Epoch:{}/100".format(epoch))
    run_epoch(word_batch_gen(word_trainset, 32,word_max_len+1,device), word_model, 
              SimpleLossCompute(word_model.generator, criterion, model_opt))
    word_model.eval()
    print("Loss:{}".format(run_epoch(word_batch_gen(word_valset, 32,word_max_len+1,device), word_model, 
                    SimpleLossCompute(word_model.generator, criterion, None))))

Epoch:0/100
Loss:0.4562869369983673
Epoch:1/100
Loss:0.352128803730011
Epoch:2/100
Loss:0.297194242477417
Epoch:3/100
Loss:0.2755328416824341
Epoch:4/100
Loss:0.24350301921367645
Epoch:5/100
Loss:0.22940757870674133
Epoch:6/100
Loss:0.2185177206993103
Epoch:7/100
Loss:0.19691374897956848
Epoch:8/100
Loss:0.19372770190238953
Epoch:9/100
Loss:0.17999398708343506
Epoch:10/100
Loss:0.16983895003795624
Epoch:11/100
Loss:0.1605202704668045
Epoch:12/100
Loss:0.15287137031555176
Epoch:13/100
Loss:0.14302320778369904
Epoch:14/100
Loss:0.13540151715278625
Epoch:15/100
Loss:0.13180458545684814
Epoch:16/100
Loss:0.12848496437072754
Epoch:17/100
Loss:0.12116862833499908
Epoch:18/100
Loss:0.11834833770990372
Epoch:19/100
Loss:0.1176949292421341
Epoch:20/100
Loss:0.1087062731385231
Epoch:21/100
Loss:0.10414064675569534
Epoch:22/100
Loss:0.10136117786169052
Epoch:23/100
Loss:0.10076241940259933
Epoch:24/100
Loss:0.09594300389289856
Epoch:25/100
Loss:0.09148602932691574
Epoch:26/100
Loss:0.090548723936

In [80]:
def word_decode(word_vec):
    ret=""
    for w in word_vec:
        if w>1:
            ret+=chr(w+ord('a')-2)
    return ret

c=0
examples=0
random.shuffle(vocab_val)
for i,word in enumerate(vocab_val):
    if i>1000:
        break
    orig_word,word_noise=word_embed(word,word_max_len+1)
    src = Variable(word_noise.unsqueeze(0), requires_grad=False)
    src_mask = Variable(torch.ones(1, 1, word_max_len+1))
    out=greedy_decode(word_model, src.to(device), src_mask.to(device), sequence_len=word_max_len, start_symbol=0).cpu()
    # cut the first element because it's the start_value
    out=(out.squeeze())[1:]
    # decode
    orig=word_decode(orig_word.numpy())
    noise=word_decode(word_noise.numpy())
    output=word_decode(out.numpy())
    if orig==output:
        c+=1
        examples+=1
        if examples<=5:
            print("Original: {} , Noise: {} , Correction: {}.".format(orig,noise,output))
print("Correction accuracy:{}".format(c/1000))

Original: hereby , Noise: hhereby , Correction: hereby.
Original: traynor , Noise: traynnor , Correction: traynor.
Original: honesty , Noise: honeesty , Correction: honesty.
Original: skyping , Noise: skypiing , Correction: skyping.
Original: adornment , Noise: adoornment , Correction: adornment.
Correction accuracy:0.289


In [54]:
"""
Problem 2.3: Spelling correction, sentence level
"""

def new_char_encode(c):
    # encode char a-z to 3->29, the others: 0 for start_value, 1 for padding,2 for space
    if c==' ':
        return 2
    return ord(c)-ord('a')+3

def create_sentence_set(dataset):
    # Create sentence_set from questions in the dataset
    sentence_set=[]
    sentence_max_len=0
    for i,each in enumerate(dataset):
        sentence=preprocess(each['question'])
        sentence_vec=[]
        start=0
        while(sentence[start]>'z' or sentence[start]<'a'):   # sentence should start with letters
            start+=1
        sentence=sentence[start:]
        # only take sentences that have length <=100
        if len(sentence)>100:
            continue
            
        for c in sentence:
            # only take the English letter and space
            if (c>'z' or c<'a') and c!=" ":
                continue
            sentence_vec.append(new_char_encode(c))
        # get the length of longest sentence
        if len(sentence_vec)>sentence_max_len:
            sentence_max_len=len(sentence_vec)
        sentence_set.append(sentence_vec)
    return sentence_set,sentence_max_len

sentence_set,sentence_max_len=create_sentence_set(squad_dataset['train'])
print(len(sentence_set),sentence_max_len)

83809 100


In [55]:
def sentence_encode(sentence,sentence_len):
    padded_sentence=[]
    padded_noise_sentence=[]
    idx=random.randint(0,len(sentence)-1)
    while(sentence[idx]==2): # random choose position of character to make noise, but not the position of spaces in sentence
        idx=random.randint(0,len(sentence)-1)     
    for i in range(sentence_len):
        if i<len(sentence):
            if i==idx:   # duplicate the character at idx position
                padded_noise_sentence.append(sentence[i])
            padded_noise_sentence.append(sentence[i])
            padded_sentence.append(sentence[i])
        else:    # padded the sentence to have length sentence_len
            padded_sentence.append(1)
            padded_noise_sentence.append(1)
    padded_noise_sentence.pop()   # noise_sentence has 1 more char because of duplicating, so we drop the last character.
                                # because the sentence_len will be passed as max_sequence_len+1, so the dropped char will be the padding
    return torch.LongTensor(padded_sentence),torch.LongTensor(padded_noise_sentence)

In [56]:
random.shuffle(sentence_set)
# divide train-val with 8:2 ratio
sentence_trainset,sentence_valset=sentence_set[:4*len(sentence_set)//5],sentence_set[4*len(sentence_set)//5:]
trainset=[] # list of training data, each data have [sentence,sentence_with_noise]
valset=[]   # list of validation data
for each in sentence_trainset:
    sentence,sentence_noise=sentence_encode(each,sentence_max_len+1)
    trainset.append([sentence,sentence_noise])
for each in sentence_valset:
    sentence,sentence_noise=sentence_encode(each,sentence_max_len+1)
    valset.append([sentence,sentence_noise])

In [57]:
def sentence_batch_gen(sentence_set, batch,sentence_len,device):
    # shuffle set for each batch
    random.shuffle(sentence_set)
    n_batches=len(sentence_set)//batch
    for batch_idx in range(n_batches):
        st=torch.LongTensor()   # tensor hold tgt
        st_noise=torch.LongTensor()  # tensor hold src
        for i in range(batch_idx*batch,batch*(batch_idx+1)):
            st=torch.cat((st,sentence_set[i][0].unsqueeze(0)),0)
            st_noise=torch.cat((st_noise,sentence_set[i][1].unsqueeze(0)),0)   
        src = Variable(st_noise, requires_grad=False)
        start_value=torch.zeros((batch,1),dtype=torch.long)
        tgt = Variable(torch.cat((start_value,st),1), requires_grad=False)
        yield Batch(src.to(device), tgt.to(device), 0)

In [78]:
V=29 # a->z +3
criterion = LabelSmoothing(size=V, padding_idx=0, smoothing=0.0)
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device=torch.device('cpu')
st_model = make_model(V, V, N=2)
st_model.to(device)
model_opt = NoamOpt(st_model.src_embed[0].d_model, 1, 400,
        torch.optim.Adam(st_model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
for epoch in range(20):
    st_model.train()
    print("Epoch:{}/20".format(epoch))
    run_epoch(sentence_batch_gen(trainset, 32,sentence_max_len+1,device), st_model, 
              SimpleLossCompute(st_model.generator, criterion, model_opt))
    st_model.eval()
    print("Loss:{}".format(run_epoch(sentence_batch_gen(valset, 32,sentence_max_len+1,device), st_model, 
                    SimpleLossCompute(st_model.generator, criterion, None))))

Epoch:0/20
Loss:0.012313617393374443
Epoch:1/20
Loss:0.0059689865447580814
Epoch:2/20
Loss:0.00498791690915823
Epoch:3/20
Loss:0.004229368641972542
Epoch:4/20
Loss:0.00335776899009943
Epoch:5/20
Loss:0.0031558291520923376
Epoch:6/20
Loss:0.0031045828945934772
Epoch:7/20
Loss:0.002857277635484934
Epoch:8/20
Loss:0.002632071264088154
Epoch:9/20
Loss:0.0024712353479117155
Epoch:10/20
Loss:0.0023817841429263353
Epoch:11/20
Loss:0.002263597911223769
Epoch:12/20
Loss:0.0022189118899405003
Epoch:13/20
Loss:0.002121024765074253
Epoch:14/20
Loss:0.0020761550404131413
Epoch:15/20
Loss:0.0020523194689303637
Epoch:16/20
Loss:0.002005132846534252
Epoch:17/20
Loss:0.001960025168955326
Epoch:18/20
Loss:0.0018159677274525166
Epoch:19/20
Loss:0.0018209557747468352


In [79]:
def sentence_decode(sentence_vec):
    ret=""
    for c in sentence_vec:
        if c>2:
            ret+=chr(c+ord('a')-3)
        elif c==2:
            ret+=" "
    return ret

random.shuffle(sentence_valset)
examples=0
for i,st in enumerate(sentence_valset):
    if i >100:
        break
    sentence=sentence_decode(st)
    words=sentence.split()
    #get a random word from the sentence val set
    word_idx=random.randint(0,len(words)-1)
    target_word=words[word_idx]
    # create noise and correct using word model
    orig_word,word_noise=word_embed(target_word,word_max_len+1)
    src = Variable(word_noise.unsqueeze(0), requires_grad=False)
    src_mask = Variable(torch.ones(1, 1, word_max_len+1))
    out=greedy_decode(word_model, src.to(device), src_mask.to(device), sequence_len=word_max_len, start_symbol=0).cpu()
    # cut the first element because it's the start_value
    out=(out.squeeze())[1:]
    # decode
    orig_w=word_decode(orig_word.numpy())
    noise_w=word_decode(word_noise.numpy())
    output_w=word_decode(out.numpy())

    # reconstruct the sentence that contains the noise word
    word_noise=word_noise.numpy()
    rec_word=word_decode(word_noise)
    words[word_idx]=rec_word
    rec_st=""
    for i in range(len(words)):
        rec_st=rec_st+words[i]+" "
    encoded_st=[]
    for i in range(len(rec_st)):
        encoded_st.append(new_char_encode(rec_st[i]))
    # now correct using sentence model
    orig_st,_=sentence_encode(st,sentence_max_len+1)
    st_noise,_=sentence_encode(encoded_st,sentence_max_len+1)
    start=torch.zeros(1,1,dtype=torch.long)
    src = Variable(st_noise.unsqueeze(0), requires_grad=False)
    src_mask = Variable(torch.ones(1, 1, sentence_max_len+1))
    out=greedy_decode(st_model, src.to(device), src_mask.to(device), sequence_len=sentence_max_len, start_symbol=0).cpu()
    out=(out.squeeze())[1:]
    orig_s=sentence_decode(orig_st.numpy())
    noise_s=sentence_decode(st_noise.numpy())
    output_s=sentence_decode(out.numpy())
    
    if orig_w!=output_w and orig_s==output_s:
        print("Target word:{}".format(target_word))
        print("Correction using word model:")
        print("Original word: {} , Noise word: {} , Correction word: {}".format(orig_w,noise_w,output_w))
        print("Correction using sentence model:")
        print("Original sentence: {}".format(orig_s))
        print("Noise sentence: {}".format(noise_s))
        print("Correction sentence: {}".format(output_s))
        print()
        examples+=1
    if examples>=3:
        break
        
    

Target word:what
Correction using word model:
Original word: what , Noise word: wwhat , Correction word: hwat
Correction using sentence model:
Original sentence: what are some of the sets or ideals most school systems follow 
Noise sentence: wwhat are some of the sets or ideals most school systems follow 
Correction sentence: what are some of the sets or ideals most school systems follow 

Target word:who
Correction using word model:
Original word: who , Noise word: whoo , Correction word: how
Correction using sentence model:
Original sentence: who can permit a person to hunt wild animals 
Noise sentence: whoo can permit a person to hunt wild animals 
Correction sentence: who can permit a person to hunt wild animals 

Target word:founded
Correction using word model:
Original word: founded , Noise word: fouunded , Correction word: oufonde
Correction using sentence model:
Original sentence: who founded a monastic order in his life 
Noise sentence: who fouunded a monastic order in his lif